<a href="https://colab.research.google.com/github/9rvLU/Study_of_DoE/blob/master/study_of_doe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DoE 勉強

3つの入力を持ち、1つの出力を持つ簡単なプログラムを作成し、
DoE により出力への影響度を調査する。

## 環境設定

- Colaboratory (Python 3.6)
- package
  - rpy2

## 条件

プログラムの入力として3つの変数 A, B, C を用いる。  
A，B, C はそれぞれ、10, 50, 100 を入力するものとする。

出力は、大きければ大きいほど評価が高いとする。

In [ ]:
!pip install rpy2

## 実験のための準備プログラムの作成

実験を行うためにプログラムを用意する。

ライブラリのインポートを行う。

In [ ]:
# ライブラリのインポート
import numpy as np
import pandas as pd


使用したパラメータは以下の通り。

In [ ]:
# 実験するパラメータ
paramTable = pd.DataFrame(
    [[10,50,100],
    [10,50,100],
    [10,50,100]],
    index=['paramA','paramB','paramC'],
    columns=['small','middle','large']
)

paramTable.head()

,small,middle,large
paramA,10,50,100
paramB,10,50,100
paramC,10,50,100


R の conjoint を使用して実験するレコードを導出する。

使用したコードは以下の通り。
```
library(conjoint)

#Create list of partners
paramTable <- expand.grid(
 paramA = c("small", "middle", "large"),
 paramB = c("small", "middle", "large"),
 paramC = c("small", "middle", "large")
)

#Create orthogonal table
design_ort <- caFactorialDesign(data=paramTable, type="orthogonal")
```

出力は以下の通り。
出力を paramTable.csv として保存した。
```
> print(design_ort)
   paramA paramB paramC
2  middle  small  small
6   large middle  small
7   small  large  small
12  large  small middle
13  small middle middle
17 middle  large middle
19  small  small  large
27  large  large  large
```


テストに使用する関数を定義する。

返り値は、`A~B / C` とする。

これにより返り値の大きさへの寄与は以下のようになる。
* A : 寄与するがBより小さい
* B : 最も大きい。
* C : 逆方向に寄与する

In [ ]:
# テストに使用する関数の定義
def GetResult(A, B, C):
        return A**B / C

## 実験を行う

直交表にしたがって計算を行う。
直交表は以下の通り。

In [ ]:
# 直交表を読み込む
L8 = pd.read_csv("paramTable.csv")

L8.head(10)

FileNotFoundError: ignored

In [ ]:
# 表と値の相関を定義
param = {"small":10, "middle":50, "large":100}

直交表から文字列を取得し数値に変換する。
その後、関数に代入して結果を得る。

In [ ]:
# 結果を格納するリストの宣言
resultList = []

# 結果リストに値を代入
for record, row in L8.iterrows():
    resultList.append(GetResult(param[row[0]], param[row[1]], param[row[2]]))

# 結果リストを paramTable に追加
L8["result"] = resultList

L8.head(10)

## 結果を評価する



In [ ]:
L8["small"==train["paramA"]]

### 参考

DoE 関連
- [試験回数を減らそう（実験計画法）](https://www.placeon.jp/blog/words/words_monozukuri/experimental_design/)
- [R 実験計画法を利用した要因分析](http://nakhirot.hatenablog.com/entry/20160918/1474166690)
- [コンジョイント分析を試してみた（まずはR編）](https://web-kaizen.co.jp/2017/07/15/%E3%82%B3%E3%83%B3%E3%82%B8%E3%83%A7%E3%82%A4%E3%83%B3%E3%83%88%E5%88%86%E6%9E%90%E3%82%92%E8%A9%A6%E3%81%97%E3%81%A6%E3%81%BF%E3%81%9F%EF%BC%88%E3%81%BE%E3%81%9A%E3%81%AFr%E7%B7%A8%EF%BC%89/)